In [4]:
#Importing required libraries 
import numpy as np 
import tensorflow as tf

In [5]:
# Initializing Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

#Reading Data
data = open(r'/content/drive/My Drive/dataset.txt').read()

#Making sentences Lower case and spliting it 
corpus = data.lower().split("<eos>")
corpus = corpus[:100]

In [6]:
#Cleaning data, removing unwanted symbols.
import re
for i in range(len(corpus)):
 corpus[i] = corpus[i].replace("\n",' ')
 corpus[i] = corpus[i].replace("<eos>",' ')
 corpus[i] = re.sub("[`~!@#$+%*:().'?-].\"", ' ',corpus[i]) 

In [7]:
#Fitting data on tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku 
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]                                                                               
label = ku.to_categorical(label, num_classes=total_words)        

In [8]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import Regularizer
#Sequential Model 
model = Sequential()

#Adding layers to the model 
model.add(Embedding(total_words,240, input_length=max_sequence_len-1))  
model.add(Bidirectional(LSTM(150))) 
model.add(Dropout(0.2)) 
model.add(Dense(total_words/2, activation='relu'))
model.add(Dense(total_words, activation='softmax'))  

In [9]:
#Compiling Model 
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])  

In [10]:
from keras.callbacks import ModelCheckpoint

# Checkpoint 
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.82):
      model.save('model.h5')
      print("\nReached 80% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
#Training model over certain epoches .
model.fit(predictors, label, epochs=1,callbacks = [callbacks], verbose=1)

22/22 [==============================] - 0s 8ms/step - loss: 6.0954 - accuracy: 0.0185


In [ ]:
#to read the .json and HDF5 file from the drive
from tensorflow.keras.models import model_from_json
json_file = open('/content/drive/My Drive/jack1.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/content/drive/My Drive/model.h5')
model = loaded_model
#Feeding the trained model to predict the words
def predict(seed_text , seed = 625):
    for i in range( seed ):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=
        max_sequence_len , padding='pre')
        predicted = model.predict_classes(token_list, verbose=0 )
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        if i!=1 and i!=0:
          if i%13==0:
            seed_text+='\n'
    return seed_text


#Giving seed text for generation of the sonnet
#data = open(r'/content/drive/My Drive/dataset2.txt').read()
sonnet_14 = input()
tk=predict(sonnet_14)
d=tk.split("\n")[1:15]
for i in range(len(d)):  
  d[i] = re.sub("[,`~!@#$+%*:()'?-]", '',d[i])
h1=d.pop(0).split(" ")
if(h1[0].lower()=='d' or h1[0].lower()=='s'or h1[0].lower()=='è'):h1[0]="As"
for i in([" ".join(h1).capitalize()]+d):
  print(i)